This is a very useful guide into TF-IDF calculation and K-means clustering using Sci-Kit Learn:

https://jonathansoma.com/lede/algorithms-2017/classes/clustering/k-means-clustering-with-scikit-learn/

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer #SciKit-Learn Machine Learning Library
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize
import nltk

In [5]:
df = pd.read_csv("grid_500m_newest_og_tfidf.csv")

df
#df.to_csv("inspect_df.csv")

,OID_,FID_grid_1000m_newest_SummarizeWithin,PageName,PageNumber,Point_Count,FID_analysis_data_final,Field1,Unnamed__0,Unnamed__0_1,index,...,tags,title,url_n,date,time,year,short_filtered,char_filtered,custom_filter,custom_filter_further
0,1,4,A5,4,184,10547,10660,11466,14775,21943,...,anderson edinburghsouthsidegravestones,Anderson 1 Oct 2013 16-22_GrangeGraveyard(SWco...,https://live.staticflickr.com/5516/11295229604...,01/10/2013 00:00:00,16:22:00,2013,anderson edinburghsouthsidegravestones,anderson edinburghsouthsidegravestones,anderson edinburghsouthsidegravestones,anderson edinburghsouthsidegravestones
1,2,1,A2,1,84,14954,15084,16562,21296,31112,...,house edinburgh villa quartz rubble rockery ra...,"“Lammerburn,” Edinburgh",https://live.staticflickr.com/5705/20941618383...,16/09/2015 00:00:00,15:19:00,2015,house edinburgh villa quartz rubble rockery ra...,house edinburgh villa quartz rubble rockery ra...,house villa quartz rubble rockery rafters mmsu...,house villa quartz rubble rockery rafters mmsu...
2,3,2,A3,2,133,17260,17396,19222,24579,35930,...,s nissan micrak12 micra k1210car edinburgh edi...,PLZ8349,https://live.staticflickr.com/4799/25823614647...,14/01/2017 00:00:00,15:48:00,2017,nissan micrak12 micra k1210car edinburgh edinb...,nissan micrak micra k car edinburgh edinb...,nissan micrak micra k car edinburghcar plz door,nissan micrak micra k car edinburghcar plz door
3,4,1,A2,1,84,14701,14831,16303,20924,30567,...,castle tattoo scotland edinburgh edinburghcast...,P1010931,https://live.staticflickr.com/1478/26257687161...,15/08/2015 00:00:00,22:32:00,2015,castle tattoo scotland edinburgh edinburghcast...,castle tattoo scotland edinburgh edinburghcast...,tattoo august militarytattoo edinburghroyalmil...,tattoo august militarytattoo edinburghroyalmil...
4,5,4,A5,4,184,10421,10534,11338,14629,21747,...,carlawmartin sirthomascarlawmartin edinburghso...,Sir Thomas Carlaw Martin 22 Sep 2013 16-06_Gra...,https://live.staticflickr.com/3791/11304661733...,22/09/2013 00:00:00,16:06:00,2013,carlawmartin sirthomascarlawmartin edinburghso...,carlawmartin sirthomascarlawmartin edinburghso...,carlawmartin sirthomascarlawmartin edinburghso...,carlawmartin sirthomascarlawmartin edinburghso...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21269,21270,50,F6,50,163,19142,19287,21373,27221,39817,...,christmas2017 boxingday newhaven family lighth...,Talking to the swans,https://live.staticflickr.com/4726/38766832364...,26/12/2017 00:00:00,13:36:00,2017,christmas2017 boxingday newhaven family lighth...,christmas boxingday newhaven family lighth...,christmas boxingday newhaven family lighthouse...,christmas boxingday newhaven family lighthouse...
21270,21271,50,F6,50,163,10593,10706,11518,14849,22081,...,uploaded:by=flickrmobile flickriosapp:filter=n...,NaN,https://live.staticflickr.com/5481/10228922866...,12/10/2013 00:00:00,16:40:00,2013,uploaded flickrmobile flickriosapp filter nofi...,uploaded flickrmobile flickriosapp filter nofi...,flickrmobile flickriosapp filter nofilter,flickriosapp filter
21271,21272,51,F7,51,304,2298,2318,2481,3167,4683,...,blue plant yellow catchycolors scotland pentax...,Loader,https://live.staticflickr.com/5284/5278448751_...,18/12/2010 00:00:00,12:19:00,2010,blue plant yellow catchycolors scotland pentax...,blue plant yellow catchycolors scotland pentax...,blue plant yellow catchycolors pentax tripod t...,blue plant yellow catchycolors tripod tracks l...
21272,21273,49,F5,49,137,15209,15339,16822,21598,31489,...,autumn anna edinburgh catriona grandchildren,Last light up the Forth,https://live.staticflickr.com/5659/22610898946...,29/10/2015 00:00:00,17:16:00,2015,autumn anna edinburgh catriona grandchildren,autumn anna edinburgh catriona grandchildren,autumn anna catriona grandchildren,autumn


In [7]:
#create vectoriser - TdidfVectorizer normalises the tfidf values from 0-1
#set use_idf to 'True' so that it actually calculates the IDF part of TF-IDF (otherwise it's just TF which is bloody confusing)
#max_df = words that appear over 80% of entire corpus is missed out
#min_df = miss out on words that appear less than 5 times in corpus
# norm = 'l2' normalises the length of the documents to prepare for calculation (between two equal-length vectors) 
Vectorizer = TfidfVectorizer(lowercase = True, stop_words = 'english', use_idf = True, norm = 'l2')

#Vectors = Vectorizer.fit_transform(df['lemmatised_tags'])

#https://stackoverflow.com/questions/64743583/which-10-words-has-the-highest-tf-idf-value-in-each-document-total

#finding top 10 tfidf tags per document
X_tfidf = Vectorizer.fit_transform(df['custom_filter_further'])
#X_tfidf_array = X_tfidf.toarray()


In [9]:
#convert array to sparse matrix
Vectors_sparse = sparse.csr_matrix(X_tfidf)
Vectors_sparse

<91x21020 sparse matrix of type '<class 'numpy.float64'>'
	with 54072 stored elements in Compressed Sparse Row format>

In [10]:
similarities = cosine_similarity(Vectors_sparse)
print('pairwise dense output:\n {}\n'.format(similarities))

pairwise dense output:
 [[1.00000000e+00 5.56280595e-03 1.38689957e-02 ... 2.65794002e-02
  1.96550679e-02 1.68362283e-03]
 [5.56280595e-03 1.00000000e+00 4.75204326e-02 ... 5.55941904e-03
  4.31176101e-03 1.16869503e-04]
 [1.38689957e-02 4.75204326e-02 1.00000000e+00 ... 4.96530394e-02
  6.65835643e-03 1.45523688e-02]
 ...
 [2.65794002e-02 5.55941904e-03 4.96530394e-02 ... 1.00000000e+00
  1.93598024e-01 1.34830211e-02]
 [1.96550679e-02 4.31176101e-03 6.65835643e-03 ... 1.93598024e-01
  1.00000000e+00 4.82276117e-03]
 [1.68362283e-03 1.16869503e-04 1.45523688e-02 ... 1.34830211e-02
  4.82276117e-03 1.00000000e+00]]



In [11]:
#check shape to see if it's a matrix - and yes it is
print(similarities.shape)

list = similarities.tolist()
cosine_similarity_matrix = pd.DataFrame(list)

#cosine_similarity_matrix.to_csv("cosine_sim_matrix_500m_no_topic_model.csv")

(91, 91)


This is useful and below code draws from it: https://stackoverflow.com/questions/12118720/python-tf-idf-cosine-to-find-document-similarity